# Trabalho Prático 2

## Introdução

A resolução deste trabalho prático tem como propósito introduzir o **SageMath**, corpos finitos primos, curvas elípticas sobre esses corpos e esquemas criptográficos baseados nos mesmos.
Os principais objetivos passam por:

- Implementar um esquema _RSA_ como uma classe **Python**.
- Implementar um esquema _ECDSA_ como uma classe **Python**.
- Implementar um esquema _ECDH_ como uma classe **Python**.

Além disso, devem ser aplicadas as seguintes particularidades aos esquemas desenvolvidos:

- O esquema _RSA_ deve fornecer métodos para cifrar, decifrar, assinar e verificar uma mensagem.
- O esquema _ECDSA_ deve utilizar uma das curvas primas definidas no **FIPS186-4**.
- O esquema _ECDH_ deve utilizar curvas elíticas binárias.

O relatório está dividido em três partes, uma por cada um dos objetivos a cumprir. Além disso, está estruturado de forma a que o texto entre os _snippets_ de código seja suficientemente explicativo, sobre a implementação e desenho da solução.

**PODERÁ SER PRECISO ALGO AQUI, COMO IMPORTS DO SAGEMATH OU ASSIM**

## Esquema RSA

O objetivo desta secção passa por definir a classe **Python** que implementa o algoritmo **RSA**. Esta permitirá inicializar uma intância, fornecendo-lhe o parâmetro de segurança (tamanho de chave). Após criada a instância, a mesma permitirá:

- Cifrar uma mensagem.
- Decifrar uma mensagem previamente cifrada.
- Assinar digitalmente uma mensagem.
- Verificar uma assinatura digital de uma mensagem, previamente produzida.

In [10]:
class RSA:
    
    def __init__(self,l):
        r = random_prime(2**((l/2) + 1)/2,(2**(l/2))/2)
        p = random_prime((2*r + 1)**2, 2*r + 1)
        self.q = p * r
        phi = (p - 1) * (r - 1)
        
        k = ZZ.random_element(phi)
        while gcd(k, phi) != 1:
             k = ZZ.random_element(phi)
        self.k = k
        
        self.s = inverse_mod(self.k,phi)
    
    def encrypt(self,plaintext):
        return power_mod(plaintext,self.k,self.q)        
    
    def decrypt(self,ciphertext):
        return power_mod(ciphertext,self.s,self.q)
    
    def sign(self,message):
        return power_mod(message,self.s,self.q)        
    
    def verify(self,message,signature):
        mk = power_mod(signature,self.k,self.q)
        return mk == message

In [11]:
r = RSA(1024)
# Se a mensagem for maior do que o q, não funciona!
# Ver como fazer nesses casos
# Ver como generalizar, convertendo uma string num número inteiro para que possa ser passada qualquer mensagem
# Possivelmente teremos que cifrar em blocos e fazer padding
msg = ZZ.random_element(2**100,2**200)
print(msg)
ciphertext = r.encrypt(msg)

plaintext = r.decrypt(ciphertext)
print(plaintext)

sig = r.sign(msg)
if(r.verify(msg,sig)):
    print('OK')
else: print('Not OK!')

114741538318899128783020272867901617965692384029848676537666
114741538318899128783020272867901617965692384029848676537666
OK


## Esquema ECDSA

## Esquema ECDH

## Conclusão

## Referências